In [1]:
import pandas as pd
import numpy as np
import gc
%matplotlib inline

In [2]:
! ls -l ./input/*.csv

-rwxrwxrwx@ 1 alexandrelacheze  staff    21254426 May 24 21:34 ./input/cliente_tabla.csv
-rwxrwxrwx@ 1 alexandrelacheze  staff      109549 May 24 21:35 ./input/producto_tabla.csv
-rwxrwxrwx@ 1 alexandrelacheze  staff   251114289 May 24 21:30 ./input/test.csv
-rw-r--r--@ 1 alexandrelacheze  staff       29179 Jun  7 23:16 ./input/town_state.csv
-rwxrwxrwx@ 1 alexandrelacheze  staff  3199358223 May 24 21:29 ./input/train.csv


In [3]:
import random
MAX_NROWS = 100000

def read_sample_csv(filename, sample_size=MAX_NROWS, **kwargs):
    nrows = sum(1 for line in open(filename)) -1
    print "we got %s rows in %s" % (nrows, filename)
    if nrows < sample_size:
        print "reading everything"
        return pd.read_csv(filename, **kwargs)
    else:
        print "sampling %s rows" % sample_size
        skip = sorted(random.sample(xrange(1, nrows+1),nrows-sample_size))
        return pd.read_csv(filename, skiprows=skip, **kwargs)

In [4]:
# borrowed from https://www.kaggle.com/ericcouto/grupo-bimbo-inventory-demand/using-82-less-memory/code
types = {
    'Semana':np.uint8,
    'Agencia_ID':np.uint16,
    'Canal_ID':np.uint8,
    'Ruta_SAK':np.uint16,
    'Cliente_ID':np.uint32,
    'Producto_ID':np.uint32,
    'Demanda_uni_equil':np.uint32
}

train = read_sample_csv('./input/train.csv', usecols=types.keys(), dtype=types)

types['id'] = np.uint32
del types['Demanda_uni_equil']
test = read_sample_csv('./input/test.csv', usecols=types.keys(), dtype=types)

we got 74180464 rows in ./input/train.csv
sampling 100000 rows
we got 6999251 rows in ./input/test.csv
sampling 100000 rows


In [9]:
test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,218,11,1347,1,1263,811505,2425
1,231,11,1216,1,1625,601148,35141
2,288,10,1347,1,1152,4300673,1309
3,327,11,23669,1,1112,4606367,1216
4,394,10,3217,1,4406,2178308,32861


In [10]:
train['test'] = 0
test['test'] = 1
data = pd.concat([train, test])
del train
del test
gc.collect()

61

In [15]:
data = data.rename(columns= {
        "Agencia_ID": "agency_id",
        "Canal_ID": "canal_id",
        "Cliente_ID": "client_id",
        "Producto_ID": "product_id",
        "Ruta_SAK": "route_id",
        "Semana": "week",
        "Demanda_uni_equil": "demand"
        })

In [106]:
def add_lagging_demand_features(data, max_week=6, inplace=False):
    if inplace == False:
        data = data.copy()

    lag_cols = []
    demand_per_CPW = data.groupby(["client_id", "product_id", "week"])["demand"].sum()

    for i in range(1, max_week+1):

        def get_lagging_demand(row):
            client_id, product_id = row["client_id"], row["product_id"]
            week = row["week"] - i
            
            if week <3:
                # we don't have anything before week 3
                return pd.np.NaN
            
            if (client_id, product_id, week) in demand_per_CPW.index:
                return demand_per_CPW.loc[client_id, product_id, week]
            else:
                # we assume we got 0 demand for this product, client
                return 0
    
        col_name = "demand_lag%s" % i
        data[col_name] = data.apply(get_lagging_demand, axis=1)
        lag_cols.append(col_name)
    
    data["demand_lagTotal"] = 0
    for col in lag_cols:
        data["demand_lagTotal"] += data[col]
    lag_cols.append("demand_lagTotal")
        
    return data, lag_cols

def add_frequency_features(data, inplace=False):
    if inplace == False:
        data = data.copy()
        
    freq_cols = []
    
    for aggr_level in ["agency", "route", "client", "product"]:
        aggr_col = aggr_level+"_id"
        new_col_name = aggr_level+"_demand"
        
        demand_per_aggr = data.groupby(aggr_col)["demand"].mean().rename(new_col_name).reset_index()
        data = pd.merge(data, demand_per_aggr, on=aggr_level+"_id",  how='left')
        data[new_col_name].fillna(0, inplace=True)
        freq_cols.append(new_col_name)
    
    return data, freq_cols


def add_frequency_lagging_features(data, inplace=True, max_week=6):
    if inplace == False:
        data = data.copy()

    freq_lag_cols = []
        
    for aggr_level in ["agency", "route", "client", "product"]:
        aggr_col = aggr_level+"_id"
        new_col_name = aggr_level+"_demand"
        
        demand_per_aggr_and_week = data.groupby([aggr_col, 'week'])["demand"].mean().rename(new_col_name).fillna(0)
        lag_cols = []
        for i in range(1, max_week+1):

            def get_lagging_demand(row):
                aggr_index = row[aggr_col]
                week = row["week"] - i
            
                if week <3:
                    # we don't have anything before week 3
                    return pd.np.NaN
            
                if (aggr_index, week) in demand_per_aggr_and_week.index:
                    return demand_per_aggr_and_week.loc[aggr_index, week]
                else:
                    # we assume we got 0 demand for this product, client
                    return 0
    
            col_name = "%s_lag%s" % (new_col_name, i)
            data[col_name] = data.apply(get_lagging_demand, axis=1)

            lag_cols.append(col_name)
            freq_lag_cols.append(col_name)
            
        lagTotal_col = "%s_lagTotal" % new_col_name
        data[lagTotal_col] = 0
        for col in lag_cols:
            data[lagTotal_col] += data[col]
        freq_lag_cols.append(lagTotal_col)
        
        
    return data, freq_lag_cols
    
def extract_features(data, inplace=False, max_lagging_shift=6):
    features_columns = ['agency_id','canal_id','client_id','product_id','route_id']
    
    # add lagging columns features
    data, lag_cols = add_lagging_demand_features(data, inplace=inplace, max_week=max_lagging_shift)
    features_columns += lag_cols
    
    # add frequency features
    data, freq_cols = add_frequency_features(data, inplace=inplace)
    features_columns += freq_cols
    
    # add frequency lagging features
    data, freq_lag_cols = add_frequency_lagging_features(data, inplace=inplace, max_week=max_lagging_shift)
    features_columns += freq_lag_cols
    
    return data, features_columns

In [53]:
def train_test_split(data, target_week=10, weeks_used=[7,8,9]):
    train = data[data.week.isin(weeks_used)]
    test = data[data.week == target_week]
    return train, test

In [107]:
pData, features_columns = extract_features(data, max_lagging_shift=3)
pData["log_demand"] = np.log1p(pData["demand"])
train, test = train_test_split(pData, target_week=9, weeks_used=[6,7,8])

X_train = train[features_columns]
X_test = test[features_columns]
y_train = train["log_demand"]
y_test = test["log_demand"]

In [108]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41838 entries, 2 to 199998
Data columns (total 29 columns):
agency_id                  41838 non-null uint16
canal_id                   41838 non-null uint8
client_id                  41838 non-null uint32
product_id                 41838 non-null uint32
route_id                   41838 non-null uint16
demand_lag1                41838 non-null float64
demand_lag2                41838 non-null float64
demand_lag3                41838 non-null float64
demand_lagTotal            41838 non-null float64
agency_demand              41838 non-null float64
route_demand               41838 non-null float64
client_demand              41838 non-null float64
product_demand             41838 non-null float64
agency_demand_lag1         41838 non-null float64
agency_demand_lag2         41838 non-null float64
agency_demand_lag3         41838 non-null float64
agency_demand_lagTotal     41838 non-null float64
route_demand_lag1          41838 non-null floa

In [109]:
# model
from sklearn import linear_model, ensemble, svm, naive_bayes
from sklearn import metrics

#clf = linear_model.RidgeCV(alphas=[0.1, 1.0, 10.0])
#clf = svm.LinearSVR()
clf = ensemble.RandomForestRegressor(n_jobs=-1)
#clf = ensemble.GradientBoostingRegressor()
#clf = naive_bayes.GaussianNB()

In [110]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print (np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.21882849609


In [95]:
import xgboost as xgb
xlf = xgb.XGBRegressor(max_depth=10, 
                        learning_rate=0.1, 
                        n_estimators=10, 
                        silent=True, 
                        objective='reg:linear', 
                        nthread=-1, 
                        gamma=0,
                        min_child_weight=1, 
                        max_delta_step=0, 
                        subsample=0.85, 
                        colsample_bytree=0.7, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=1, 
                        scale_pos_weight=1, 
                        seed=1440, 
                        missing=None)

xlf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)], early_stopping_rounds=500)

# calculate the auc score
y_pred = xlf.predict(X_test)
print (np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Will train until validation_0 error hasn't decreased in 500 rounds.
[0]	validation_0-rmse:1.246617
[1]	validation_0-rmse:1.126006
[2]	validation_0-rmse:1.040392
[3]	validation_0-rmse:0.941423
[4]	validation_0-rmse:0.852620
[5]	validation_0-rmse:0.773191
[6]	validation_0-rmse:0.702645
[7]	validation_0-rmse:0.655367
[8]	validation_0-rmse:0.597748


0.546391734938


[9]	validation_0-rmse:0.546391


In [156]:
# let's predict week 10
prep_data10, features_columns10 = extract_features(data, max_lagging_shift=3)
prep_data10["log_demand"] = np.log1p(prep_data10["demand"])
train10, test10 = train_test_split(prep_data10, target_week=10, weeks_used=[6,7,8,9])

X_train10 = train10[features_columns10]
X_test10 = test10[features_columns10]
y_train10 = train10["log_demand"]

clf10 = ensemble.RandomForestRegressor(n_jobs=-1)
clf10.fit(X_train10, y_train10)
y_pred10 = clf10.predict(X_test10)
test10['demand_pred'] = np.exp(y_pred10)-1

# add the prediction for week 10 to the data (data2 = data with prdiction of week 10)
data2 = pd.merge(data, test10[['id', 'demand_pred']], on='id', how='left')
data2['demand'] = data2.apply(lambda row: row['demand_pred'] if row['week'] == 10 else row['demand'],axis=1)
data2.drop('demand_pred', axis=1, inplace=True)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ValueError: labels ['demand_pred'] not contained in axis

In [160]:
# let's now go over again and predict week 11 with the data with have now (ie with week 10)\
prep_data11, features_columns11 = extract_features(data2, max_lagging_shift=4)
prep_data11["log_demand"] = np.log1p(prep_data11["demand"])
train11, test11 = train_test_split(prep_data11, target_week=11, weeks_used=[7,8,9, 10])

X_train11 = train11[features_columns11]
X_test11 = test11[features_columns11]
y_train11 = train11["log_demand"]

clf11 = ensemble.RandomForestRegressor(n_jobs=-1)
clf11.fit(X_train11, y_train11)
y_pred11 = clf11.predict(X_test11)
test11['demand_pred'] = np.exp(y_pred11)-1

# add the prediction for week 11 to the data (data3 = data with prdiction of week and 11)
data3 = pd.merge(data2, test11[['id', 'demand_pred']], on='id', how='left')
data3['demand'] = data3.apply(lambda row: row['demand_pred'] if row['week'] == 11 else row['demand'], axis=1)
data3.drop('demand_pred', axis=1, inplace=True)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [161]:
submission = data3[data3.test == 1][['id', 'demand']]
submission.to_csv('./output/submission.csv', index=False)

In [162]:
submission

,id,demand
100000,218.0,0.000000
100001,231.0,0.000000
100002,288.0,1.000000
100003,327.0,0.000000
100004,394.0,0.000000
100005,473.0,0.000000
100006,481.0,0.000000
100007,491.0,0.000000
100008,590.0,0.000000
100009,619.0,0.000000
